In [1]:
import pandas as pd
import datetime
import spotipy
from secrets import secrets
from sqlalchemy import create_engine
from spotipy.oauth2 import SpotifyOAuth

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=secrets["SPOTIFY_CLIENT_ID"],
                                               client_secret=secrets["SPOTIFY_CLIENT_SECRET"],
                                               redirect_uri="http://localhost:8892/callback",
                                               scope="user-read-recently-played user-top-read"))

In [3]:
RANGES = {'short_term': 'Short', 'medium_term': 'Medium', 'long_term': 'Long'}
LIMIT = 50

def get_user_df(sp):
    user = sp.me()
    df_user = pd.DataFrame({
        'user_id': user['id'],
        'display_name': user['display_name'],
        'spotify_url': user['external_urls']['spotify'],
        'image_url': user['images'][0]['url'],
        'followers': user['followers']['total'],
        'last_login': datetime.datetime.now()
    }, index=[0])
    return df_user

def get_recently_played_df(sp):
    user_id = sp.me()['id']
    recently_played = sp.current_user_recently_played()
    dict_list = []
    for rp in recently_played['items']:
        this_track = {
            'user_id': user_id,
            'track': rp['track']['name'],
            'artists': "; ".join(a['name'] for a in rp['track']['artists']),
            'album': rp['track']['album']['name'],
            'track_url': rp['track']['external_urls']['spotify'],
            'played_at': rp['played_at']
        }
        dict_list.append(this_track)
    return pd.DataFrame.from_dict(dict_list)

def get_current_playlists_df(sp):
    user_id = sp.me()['id']
    user_playlists = sp.current_user_playlists()
    dict_list = []
    for pl in user_playlists['items']:
        this_playlist = {
            'user_id': user_id,
            'playlist_id': pl['id'],
            'name': pl['name'],
            'description': pl['description'],
            'owner_display_name': pl['owner']['display_name'],
            'playlist_url': pl['external_urls']['spotify'],
            'track_numbers': pl['tracks']['total']
        }
        dict_list.append(this_playlist)
    return pd.DataFrame.from_dict(dict_list)

def get_top_artists_df(sp):
    user_id = sp.me()['id']
    dict_list = []
    for r in RANGES:
        top_artists = sp.current_user_top_artists(time_range=r, limit=LIMIT)
        for i, a in enumerate(top_artists['items']):
            this_artist = {
                'user_id': user_id,
                'rank': i+1,
                'artist': a['name'],
                'genres': "; ".join(g for g in a['genres']),
                'artist_url': a['external_urls']['spotify'],
                'timeframe': RANGES[r]
            }
            dict_list.append(this_artist)
    return pd.DataFrame.from_dict(dict_list)

def get_top_tracks_df(sp):
    user_id = sp.me()['id']
    dict_list = []
    for r in RANGES:
        top_tracks = sp.current_user_top_tracks(time_range=r, limit=LIMIT)
        for i, t in enumerate(top_tracks['items']):
            this_track = {
                'user_id': user_id,
                'rank': i+1,
                'track': t['name'],
                'artists': "; ".join(a['name'] for a in t['artists']),
                'album': t['album']['name'],
                'track_url': t['external_urls']['spotify'],
                'timeframe': RANGES[r]
            }
            dict_list.append(this_track)
    return pd.DataFrame.from_dict(dict_list)

In [4]:
df_user = get_user_df(sp)
df_user.head()

,user_id,display_name,spotify_url,image_url,followers,last_login
0,12120382831,Bin Xuan Kong,https://open.spotify.com/user/12120382831,https://scontent-hkt1-2.xx.fbcdn.net/v/t1.0-1/...,53,2020-12-25 18:27:52.311326


In [5]:
df_rp = get_recently_played_df(sp)
df_rp.head()

,user_id,track,artists,album,track_url,played_at
0,12120382831,"You're Gonna Go Far, Kid",The Offspring,"Rise And Fall, Rage And Grace",https://open.spotify.com/track/6TfBA04WJ3X1d1w...,2020-12-25T10:26:03.455Z
1,12120382831,Always,blink-182,Greatest Hits,https://open.spotify.com/track/4B7e9sBkcw1iWJ4...,2020-12-25T10:23:05.138Z
2,12120382831,Basket Case,Green Day,Dookie,https://open.spotify.com/track/6L89mwZXSOwYl76...,2020-12-25T10:18:46.925Z
3,12120382831,I Miss You,blink-182,blink-182,https://open.spotify.com/track/1AdYZ6X00nXmO61...,2020-12-25T10:15:45.006Z
4,12120382831,The Dogs,Hiroyuki Sawano,"TV Anime ""Attack on Titan Season 2"" (Original ...",https://open.spotify.com/track/41FVSWUDLLOQok5...,2020-12-25T10:07:52.036Z


In [6]:
df_cp = get_current_playlists_df(sp)
df_cp.head()

,user_id,playlist_id,name,description,owner_display_name,playlist_url,track_numbers
0,12120382831,3EE6g5tgJxg4dh1iLBjszN,rewind,for the memories,Bin Xuan Kong,https://open.spotify.com/playlist/3EE6g5tgJxg4...,45
1,12120382831,1yiei0loSfJ5Dw6V8FX3HK,gang gang gang,for the turnt,Bin Xuan Kong,https://open.spotify.com/playlist/1yiei0loSfJ5...,44
2,12120382831,1lCAKTr58quQSN3kG3rVVc,good vibes only,for the mood,Bin Xuan Kong,https://open.spotify.com/playlist/1lCAKTr58quQ...,46
3,12120382831,1tQnCB9tYc43xuiC7i6odH,s a d b o y s,for the nights,Bin Xuan Kong,https://open.spotify.com/playlist/1tQnCB9tYc43...,46
4,12120382831,7c0J78KSxxDrADgG0BpJU9,feelin blue,for the sad,Bin Xuan Kong,https://open.spotify.com/playlist/7c0J78KSxxDr...,42


In [7]:
df_ta = get_top_artists_df(sp)
df_ta.head()

,user_id,rank,artist,genres,artist_url,timeframe
0,12120382831,1,Jay Chou,c-pop; mandopop; taiwan pop; zhongguo feng,https://open.spotify.com/artist/2elBjNSdBE2Y3f...,Short
1,12120382831,2,Kanye West,chicago rap; rap,https://open.spotify.com/artist/5K4W6rqBFWDnAN...,Short
2,12120382831,3,Kid Cudi,ohio hip hop; pop rap; rap,https://open.spotify.com/artist/0fA0VVWsXO9YnA...,Short
3,12120382831,4,Joji,alternative r&b; viral pop,https://open.spotify.com/artist/3MZsBdqDrRTJih...,Short
4,12120382831,5,The Weeknd,canadian contemporary r&b; canadian pop; pop,https://open.spotify.com/artist/1Xyo4u8uXC1ZmM...,Short


In [8]:
df_tt = get_top_tracks_df(sp)
df_tt.head()

,user_id,rank,track,artists,album,track_url,timeframe
0,12120382831,1,安靜,Jay Chou,范特西,https://open.spotify.com/track/4FOydQ5IdqmdeBg...,Short
1,12120382831,2,Collard Greens,ScHoolboy Q; Kendrick Lamar,Oxymoron (Deluxe),https://open.spotify.com/track/0zO8ctW0UiuOefR...,Short
2,12120382831,3,With You (Jai Wolf Remix),Dirty South; FMLYBND,With You (The Remixes),https://open.spotify.com/track/5H2cgEPEJxA72Bq...,Short
3,12120382831,4,All Day,Kanye West; Theophilus London; Allan Kingdom; ...,All Day,https://open.spotify.com/track/79reBHqpRoWXaQb...,Short
4,12120382831,5,Nikes,Frank Ocean,Blonde,https://open.spotify.com/track/19YKaevk2bce4od...,Short


In [9]:
# TO SQL
engine = create_engine(secrets['DATABASE_URL'])
df_user.to_sql('Users', engine, index=False, if_exists='replace')
df_rp.to_sql('RecentlyPlayed', engine, index=False, if_exists='replace')
df_cp.to_sql('CurrentPlaylists', engine, index=False, if_exists='replace')
df_ta.to_sql('TopArtists', engine, index=False, if_exists='replace')
df_tt.to_sql('TopTracks', engine, index=False, if_exists='replace')

In [10]:
engine.dispose()